In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('~/Dropbox/Element/Phase2/TradesJan25.csv')

In [10]:
df.columns

Index(['fillNumber', 'version', 'execEngineType', 'sysEnvironment',
       'runStatus', 'dataCenter', 'execStatus', 'execShape', 'packageId',
       'parentNumber',
       ...
       'minTickSize', 'priceFormat', 'uPriceFormat', 'liveUPrc', 'liveMark',
       'srcTimestamp', 'sgwTimestamp', 'engTimestamp', 'timestamp',
       'timestamp_us'],
      dtype='object', length=244)

In [4]:
# Drop zero fill rows
df = df[df['fillQuantity'] > 0]

In [5]:
# Deal with apparent float precision issues to return true penny prices
round_cols = [col for col in df.columns[3:] if df[col].dtype=='float64']
round_cols = [col for col in round_cols if ('Vol' not in col) and ('Prob' not in col) and ('Mark' not in col)]
for col in round_cols:
    df[col] = df[col].apply(lambda x: round(x, 2))

In [6]:
# Convert Date Cols and add in microsecond cols
def col_to_time(col):
    df[col] = df[col].apply(pd.to_datetime)
    df[col] = df[col].dt.tz_localize('America/Chicago').dt.tz_convert('America/New_York')
    s = df[col+'_us'].apply(pd.Timedelta, unit='micros')
    df[col] = df[col] + s
    
for col in ['childDttm', 'fillTransactDttm']:
    col_to_time(col)

df['parentDttm'] = df['parentDttm'].apply(pd.to_datetime)
df['parentDttm'] = df['parentDttm'].dt.tz_localize('America/Chicago').dt.tz_convert('America/New_York')

In [7]:
parents = df['baseParentNumber'].unique()
parents

array([1136625940034889922,  763530857469211635])

In [9]:
order1 = df[df['baseParentNumber'] == parents[0]] # option
order2 = df[df['baseParentNumber'] == parents[1]] # stock hedge

In [10]:
for o in [order1, order2]:
    print(o['parentDttm'].unique())

<DatetimeArray>
['2021-01-25 10:09:42-05:00']
Length: 1, dtype: datetime64[ns, America/New_York]
<DatetimeArray>
['2021-01-25 10:09:44-05:00', '2021-01-25 10:10:03-05:00',
 '2021-01-25 10:10:06-05:00', '2021-01-25 10:10:13-05:00',
 '2021-01-25 10:10:33-05:00', '2021-01-25 10:10:34-05:00',
 '2021-01-25 10:11:02-05:00', '2021-01-25 10:11:08-05:00',
 '2021-01-25 10:11:11-05:00', '2021-01-25 10:33:42-05:00',
 '2021-01-25 10:33:44-05:00', '2021-01-25 10:57:38-05:00',
 '2021-01-25 11:21:36-05:00', '2021-01-25 11:45:47-05:00',
 '2021-01-25 11:46:39-05:00', '2021-01-25 11:46:43-05:00',
 '2021-01-25 11:46:45-05:00', '2021-01-25 11:46:46-05:00',
 '2021-01-25 11:46:48-05:00', '2021-01-25 11:46:49-05:00',
 '2021-01-25 11:46:55-05:00', '2021-01-25 11:46:56-05:00',
 '2021-01-25 12:10:14-05:00', '2021-01-25 12:12:35-05:00',
 '2021-01-25 12:12:53-05:00', '2021-01-25 12:13:21-05:00',
 '2021-01-25 12:13:25-05:00', '2021-01-25 12:13:28-05:00',
 '2021-01-25 12:33:27-05:00', '2021-01-25 12:33:33-05:00',
 '

In [11]:
def calc_fil_pctSpd(df):
    # Returns average fill as % of spread at arrival time.  0 = Bid, 0.5 = Mid, 1 = Ask
    s = (df['fillPrice'] - df['fillBid']) / (df['fillAsk'] - df['fillBid']) * df['fillQuantity']
    return s.sum() / df['fillQuantity'].sum()

for o in [order1, order2]:
    print(f'{calc_fil_pctSpd(o):.2%}')
    
# Ok.  'Active taker algos' only ever hit the bid or offer - they just think they're smart about when
# The 'patient' algos seem to do very little making

100.00%
0.20%


In [ ]:
# Compare delta adjusted fill vs arrival mid and mark
arrival_ul = (order3.loc[order3.index[0], 'parentAsk'] + order3.loc[order3.index[0], 'parentBid'])/2
arrival_idx = (order2.loc[order2.index[0], 'parentUAsk'] + order2.loc[order2.index[0], 'parentUBid'])/2
arrival_mid = (order2.loc[order2.index[0], 'parentAsk'] + order2.loc[order2.index[0], 'parentBid'])/2
arrival_mark = order2.loc[order2.index[0], 'parentMark']

print(arrival_ul)
print(arrival_idx)
print(arrival_mid)
print(arrival_mark)

In [ ]:
# Delta adjust option fill by actual SPY execution
avg_ul = (order3['fillPrice'] * order3['fillQuantity']).sum() / order3['fillQuantity'].sum()
avg_opt = (order2['fillPrice'] * order2['fillQuantity']).sum() /  order2['fillQuantity'].sum() 
delta = order2.loc[order2.index[0], 'fillDe']
delta_exec = order3['fillQuantity'].sum() / (order2['fillQuantity'].sum() * 100 * 10)
print(delta, delta_exec)
arrival_opt_adj = avg_opt + delta * (arrival_ul - avg_ul)
print(avg_opt)
print(arrival_opt_adj)
print (arrival_opt_adj - arrival_mid)
print (arrival_opt_adj - arrival_mark)

In [ ]:
# Compare vs contract vega
vega = order2.loc[order2.index[0], 'fillVe']
print(vega)
print((arrival_opt_adj - arrival_mid) / vega)
print((arrival_opt_adj - arrival_mark) / vega)
# so less than .1 vol discount vs. mark

In [ ]:
# Compare vs (*not* fill-weighted) average spread
spd_sum = (order2['fillAsk'] - order2['fillBid']).sum()
half_spd = 0.5 * spd_sum / order2.shape[0]
print(half_spd)
print((arrival_opt_adj - arrival_mid) / half_spd)
print((arrival_opt_adj - arrival_mark) / half_spd)

# This doesn't look good.  It means we crossed spread on both options and delta
# so ended up selling below the bid

# Although - we only capture the bid-offer when it trades.  That might be when it's unusually tight

# Also, this analysis doesn't take into account whether vol went down over the period.
# We could use the ATM vols provided to investigate that

In [ ]:
# Calculate theoretical delta adj as if exactly hedging each fill
# pctCross is how much of spread to assume is crossed. 0 = bid, 1 = ask

pctCross = 0.5
ul_prices = (1 - pctCross) * order2['fillUBid'] + pctCross * order2['fillUAsk']
avg_theo_ul = (ul_prices * order2['fillQuantity']).sum() / order2['fillQuantity'].sum()
arrival_opt_theo_adj = avg_opt + delta * (arrival_idx - avg_theo_ul)
arrival_opt_theo_adj

# Interesting.  It seems that SPY is so tight and hedging so fast that it makes no difference
# The execution price is dominated by the option fills

In [ ]:
# For the stock trades, how many were making vs taking?

def calc_post_pct(df):
    post_shrs = df.loc[df['childOrderHandling']=='PostLimit', 'fillQuantity'].sum()
    return post_shrs / df['fillQuantity'].sum()

print(f'{calc_post_pct(order1):.1%}') # TwapAlpha
print(f'{calc_post_pct(order3):.1%}') # Mix of AlphaVwap2pct and SpdrAuto

In [ ]:
# An observation:
half_spd_SPY = 0.5 * (order1['fillAsk'] - order1['fillBid']).sum() / order1.shape[0]
opt_cost = half_spd * order2['fillQuantity'].sum() * 100
spy_cost = half_spd_SPY * order3['fillQuantity'].sum()
print(f'SPY hedge cost is {spy_cost / (spy_cost + opt_cost):.0%} of total cost.')